In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import typing
import time

In [ ]:
def doGet(url: str) -> BeautifulSoup:
    return BeautifulSoup(requests.get(url).content, 'html.parser')

In [ ]:
# get all counties of Romania
counties_page = doGet('https://en.wikipedia.org/wiki/Counties_of_Romania')
counties_table_rows = counties_page.select_one('#mw-content-text > div.mw-parser-output > table.wikitable').find('tbody').find_all('tr')[1:]
counties_data = [(elem.text.strip() for elem in row.select('th, td')[:-2]) for row in counties_table_rows]
counties_df = pd.DataFrame(counties_data)
counties_df.rename(columns={0: 'County', 1: 'County Seat', 3: 'Development Region', 4: 'ISO Code', 7: 'NUTS Code', 8: 'Population'}, inplace=True)
counties_df['County Seat'] = [re.sub('\[.*\]', '', value) for value in counties_df['County Seat']]
counties_df.drop(columns=[2, 5, 6], inplace=True)
# easier
counties_df.loc[9]['County Seat'] = 'Bucharest'
counties_df['Population'] = [value.replace(',','') for value in counties_df['Population']]
counties_df['Population'] = pd.to_numeric(counties_df['Population'])

In [ ]:
counties_df.to_json('../data/counties.json', orient='records')